In [232]:
import pandas as pd
import numpy as np

from os import listdir
from os.path import isfile, join

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

In [233]:
datasets = {}

path = "data/"
for f in listdir(path):
    if isfile(join(path, f)):
        datasets[f.replace(".csv", "")] = pd.read_csv(join(path, f))

print(datasets.keys())

dict_keys(['clubs', 'club_games', 'game_lineups', 'players', 'stats'])


player_valuations: Les joueurs ont leur prix évalués chaque année, 2 fois, dans le dataset.
Hors, la dernière année comporte qu'une valeur. Cette valeur correspond a celle du dataset players.csv
On peut donc se passer de player_valuations.

In [234]:
# Clear columns
clear_columns = {
    "players": {
        "first_name",
        "last_name",
        "player_code",
        "city_of_birth",
        "country_of_citizenship",
        "image_url",
        "url"
    },

    "stats": {
        "Rk", "Born", 
    }
}

# Clear too old data, we keep 2023 ones.
# The issue is that, we have valuations for each year. Something we don't want.

# Some names are in double in stats.csv, keep the newer


Players et stats sont 2 datasets fusionnés
Ce qu'on peut extraire des autres csv est le ratio de victoires ( club_games ), combien de fois le joueur a été capitaine, si il était remplaçant ou titulaire ( game_lineups ),  

In [235]:
lups = datasets["game_lineups"]
all_players = lups['player_id'].unique()

# capitaines
team_captain_games = lups[lups['team_captain'] == 1]
captain_counts = team_captain_games['player_id'].value_counts()
captain_counts = captain_counts.reindex(all_players, fill_value=0)

# starting
starting_lineups = lups[lups['type'] == "starting_lineup"]
starting_counts = starting_lineups['player_id'].value_counts()
starting_counts = starting_counts.reindex(all_players, fill_value=0)

# substitutes
substitutes = lups[lups['type'] == "substitutes"]
substitutes_counts = substitutes['player_id'].value_counts()
substitutes_counts = substitutes_counts.reindex(all_players, fill_value=0)

# Rename
substitutes_counts.columns = ['player_id', 'substitutes_number']
starting_counts.columns = ['player_id', 'starting_lineup_number']
captain_counts.columns = ['player_id', 'captain_number']

starting_counts = starting_counts.reset_index()
substitutes_counts = substitutes_counts.reset_index()
captain_counts = captain_counts.reset_index()

# Merge the clean_dataset with counts DataFrames
_datasetCleaning = pd.merge(pd.merge(datasets["stats"], datasets["players"], on="name"), substitutes_counts, on="player_id", how="left")
_datasetCleaning = pd.merge(_datasetCleaning, starting_counts, on="player_id", how="left")
_datasetCleaning = pd.merge(_datasetCleaning, captain_counts, on="player_id", how="left")

In [236]:
print(_datasetCleaning.shape)

(1933, 149)


1.? - 1-Hot encoding/String transform

In [237]:
_datasetCleaning["foot"] = [int(x == "right") for x in _datasetCleaning["foot"]]
_datasetCleaning = pd.concat([_datasetCleaning, pd.get_dummies(_datasetCleaning.position)], axis=1)
_datasetCleaning = pd.concat([_datasetCleaning, pd.get_dummies(_datasetCleaning.sub_position)], axis=1)
_datasetCleaning = pd.concat([_datasetCleaning, pd.get_dummies(_datasetCleaning.Comp)], axis=1)

1.? - Nettoyage

In [238]:
# Identify non-numeric columns in the dataset
post_process_drops = [
    "name", "Nation", "first_name", "last_name", "player_code", "city_of_birth", "date_of_birth", "contract_expiration_date", "agent_name", "image_url", 
    'current_club_domestic_competition_id', "url",
    "Squad", # Comporte 98 valeurs uniques, peu utilisable avec un dataset ayant moins de 2k entrées, donc je le drop.
    "country_of_birth", "country_of_citizenship", # 102 valeurs uniques
    "sub_position", "position", "Comp", # 1-Hot encoding
    "current_club_name", # 198 valeurs uniques
    "Pos" # Doublon
]

_datasetCleaning = _datasetCleaning.drop(post_process_drops, axis=1, errors='ignore')

# Plus de valeurs avec des textes
assert(len(_datasetCleaning.select_dtypes(exclude=['number']).columns) == 0)

# Nettoyage des NaN
columnsSizePreNA = _datasetCleaning.shape[1]
_datasetCleaning.dropna(how='all', axis=1, inplace=True) # On retire les colonnes inutiles
print("On a retiré " + str(columnsSizePreNA - _datasetCleaning.shape[1]) + " colonnes.")

columnsSizePreNA = _datasetCleaning.shape[0]
_datasetCleaning.dropna(how='any', axis=0, inplace=True) # Et on retire les valeurs nulles
print("On a retiré " + str(columnsSizePreNA - _datasetCleaning.shape[0]) + " entrées.")

# Plus de NaN
assert(not _datasetCleaning.isnull().values.any())

On a retiré 3 colonnes.
On a retiré 58 entrées.


2 - Préparation du modèle

In [239]:
X = _datasetCleaning.drop('market_value_in_eur', axis=1)
Y = _datasetCleaning["market_value_in_eur"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

3 - Modèle(s)

In [240]:
models = {
    "Linear Regression": (LinearRegression(), {"model__fit_intercept": [True, False]}),
    "Ridge Regression": (Ridge(), {"model__alpha": [0.1, 1.0, 10.0]}),
    "Lasso Regression": (Lasso(), {"model__alpha": [0.01, 0.1, 1.0]}),
    "ElasticNet": (ElasticNet(), {"model__alpha": [0.01, 0.1, 1.0], "model__l1_ratio": [0.3, 0.5, 0.7]}),
    "Decision Tree": (DecisionTreeRegressor(), {"model__max_depth": [3, 5, 10], "model__min_samples_split": [2, 5, 10]}),
    "Random Forest": (RandomForestRegressor(random_state=42), {"model__n_estimators": [50, 100, 200], "model__max_depth": [3, 5, 10]}),
    "Gradient Boosting": (GradientBoostingRegressor(random_state=42), {"model__n_estimators": [50, 100, 200], "model__learning_rate": [0.01, 0.1, 0.2], "model__max_depth": [3, 5]})
}

# Dictionary to store results
results = {}

# Train and evaluate each model
for name, (model, params) in models.items():
    print(f"Training {name}...")
    
    # Create the pipeline with standard scaler and the model
    pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("model", model)
    ])
    
    # Perform GridSearchCV
    grid = GridSearchCV(
        pipeline, 
        param_grid=params, 
        cv=5, 
        scoring="neg_mean_squared_error", 
        n_jobs=-1
    )
    grid.fit(X_train, y_train)
    
    # Evaluate on test set
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Store results
    results[name] = {
        "Best Params": grid.best_params_,
        "Best CV Score (MSE)": -grid.best_score_,
        "Test MSE": mse,
        "Test R^2": r2,
        "Best Model": best_model
    }

# Print summary of results
for name, res in results.items():
    print(f"\n{name}")
    print(f"Best Params: {res['Best Params']}")
    print(f"Best CV Score (MSE): {res['Best CV Score (MSE)']:.2f}")
    print(f"Test MSE: {res['Test MSE']:.2f}")
    print(f"Test R^2: {res['Test R^2']:.2f}")

Training Linear Regression...
Training Ridge Regression...
Training Lasso Regression...


c:\Users\fabio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.144e+16, tolerance: 3.808e+13
  model = cd_fast.enet_coordinate_descent(


Training ElasticNet...


c:\Users\fabio\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.828e+16, tolerance: 3.808e+13
  model = cd_fast.enet_coordinate_descent(


Training Decision Tree...
Training Random Forest...
Training Gradient Boosting...

Linear Regression
Best Params: {'model__fit_intercept': True}
Best CV Score (MSE): 83246419591122.97
Test MSE: 90169928183479.84
Test R^2: 0.57

Ridge Regression
Best Params: {'model__alpha': 10.0}
Best CV Score (MSE): 80451899101244.53
Test MSE: 85453100994042.78
Test R^2: 0.60

Lasso Regression
Best Params: {'model__alpha': 1.0}
Best CV Score (MSE): 82034656064319.56
Test MSE: 88300104545104.47
Test R^2: 0.58

ElasticNet
Best Params: {'model__alpha': 0.1, 'model__l1_ratio': 0.5}
Best CV Score (MSE): 80091797212010.69
Test MSE: 79687622748092.45
Test R^2: 0.62

Decision Tree
Best Params: {'model__max_depth': 5, 'model__min_samples_split': 2}
Best CV Score (MSE): 70451520140074.39
Test MSE: 64239761909074.19
Test R^2: 0.70

Random Forest
Best Params: {'model__max_depth': 10, 'model__n_estimators': 100}
Best CV Score (MSE): 42210642355178.98
Test MSE: 49409652441037.11
Test R^2: 0.77

Gradient Boosting
Be